# Making searching better by using LLM(Large Language Models)

### Key Word Search

### Setup
Load needed API keys and relevant Python libaries.

In [1]:
 !pip install cohere
 !pip install weaviate-client
 !pip install python-dotenv
 !pip install umap-learn

Let's start by imporing Weaviate to access the Wikipedia database.

Weaviate is open source database that have keyword and vector search cababilities that rely on language models.

In [21]:
# the api key helps to connect with weaviate database. its like my home key
import weaviate
auth_config = weaviate.auth.AuthApiKey(
    api_key='76320a90-53d8-42bc-b4cd-678647c6672e'
)

In [23]:
client = weaviate.Client(
    url= 'https://cohere-demo.weaviate.network/',
    auth_client_secret=auth_config,
    additional_headers={
        "X-Cohere-Api-Key":'ugFJXG6voNWkQ35qwSi13dwk5ba8kSxgfUZ2UEoI',
    }
)# getting api connection problem when tring to conncect the server with client

UnexpectedStatusCodeException: Meta endpoint! Unexpected status code: 401, with response body: {'code': 401, 'message': 'invalid api key, please provide a valid api key'}.

In [ ]:
client.is_ready() # this is to check the remote database connection with this client

The high level look how `key word` search work is that it compares the number of common words between the query and the articles and gives the article that have higher number of key word.

In [24]:
                                                            # we have parameters the we pass for keyword_search function
def keyword_search(query,                                   # the query it tekes to compare with the articles
                   results_lang='en',                       # the language preference
                   properties = ["title","url","text"],     # the type of article properties we want from the result
                   num_results=3):                          # number of article we want to desplay in our result

    where_filter = {
    "path": ["lang"],
    "operator": "Equal",
    "valueString": results_lang
    }
    
    response = (
        client.query.get("Articles", properties)        # aticle with defined properties
        .with_bm25(                                     # the algorithm we use to calculate the common key words
            query=query                                 # specifing the query
        )
        .with_where(where_filter)                       # spacifing detailed spacification we want in the language
        .with_limit(num_results)                        # specifing the number of result we need to desplay
        .do()
        )

    result = response['data']['Get']['Articles']        # return the result
    return result

The query

In [25]:
query = "What is the most viewed televised event?"
keyword_search_results = keyword_search(query)
print(keyword_search_results)

C:\Users\nigus\anaconda3\lib\ast.py:50: ResourceWarning: unclosed <ssl.SSLSocket fd=384, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.0.9', 53869), raddr=('34.110.204.130', 443)>
  return compile(source, filename, mode, flags,
C:\Users\nigus\anaconda3\lib\ast.py:50: ResourceWarning: unclosed <ssl.SSLSocket fd=1648, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.0.9', 53876), raddr=('34.110.204.130', 443)>
  return compile(source, filename, mode, flags,


NameError: name 'client' is not defined

### Try modifying the search options
Other languages to try: `en, de, fr, es, it, ja, ar, zh, ko, hi`

In [26]:
properties = ["text", "title", "url", 
             "views", "lang"]

In [27]:
def print_result(result):
    """ Print results with colorful formatting """
    for i,item in enumerate(result):
        print(f'item {i}')
        for key in item.keys():
            print(f"{key}:{item.get(key)}")
            print()
        print()

In [28]:
query = "What is the most viewed televised event?"
keyword_search_results = keyword_search(query, results_lang='de')
print_result(keyword_search_results)

NameError: name 'client' is not defined

## Limitation of Key word search

Key word is not always the best choice because during queries that have no `shared or common key words` it tries to return results(articlse) that are not related with the query.